# Adding UMAP Coordinates

### Globals, imports

In [1]:
import anndata
import scvelo as scv
import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sbn
import scipy
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [7, 7]
plt.rcParams['figure.dpi'] = 440

plotting_colors = {'Apical progenitors': 'navy',
 'Intermediate progenitors':'steelblue',
 'Red blood cells': 'saddlebrown',
 'VLMC': 'gold',
 'Immature neurons': 'rebeccapurple',
 'Cajal Retzius cells': 'orangered',
 'Microglia': 'pink',
 'Migrating neurons': 'lavender',
 'Endothelial cells': 'lemonchiffon',
 'Pericytes': 'chocolate',
 'Interneurons': 'blue',
 'SCPN': 'green',
 'CThPN': 'darkgreen',
 'DL CPN': 'maroon',
 'UL CPN': 'firebrick',
 'Layer 4': 'darksalmon',
 'Oligodendrocytes': 'mediumvioletred',
 'Layer 6b': 'yellowgreen',
 'NP': 'limegreen',
 'Cycling glial cells': 'lavenderblush',
 'Astrocytes': 'darkmagenta',
 'Ependymocytes': 'blueviolet'}

In [2]:
path_to_umap = "/home/gridsan/ssauceda/neuroTF_shared/mouse_cortex_development/umap/"
umap_coords_file = path_to_umap + "cluster_scDevSC.merged.umap.txt"
adata_file = path_to_umap + "reduced_counts_adata.h5ad"

arlotta_adata = anndata.read_h5ad(adata_file)
umap_info = pd.read_csv(umap_coords_file, header=None, sep='\t')

## Formatting UMAP info

In [3]:
axes = list(umap_info.iloc[0, 1:])
barcodes = list(umap_info.iloc[2:, 0])
umap_info = umap_info.iloc[2:, 1:].astype(float)

umap_info.columns = axes
umap_info.index = barcodes

our_barcodes_ordered = arlotta_adata.obs.index
matching_barcodes = umap_info.index.isin(our_barcodes_ordered)
umap_info = umap_info[matching_barcodes]
umap_info = umap_info.reindex(our_barcodes_ordered)
umap_info = np.array(umap_info)

In [4]:
umap_info

array([[ -3.90202421, -10.86965657],
       [ -2.91512746, -12.21966172],
       [ -3.04643888, -11.65679932],
       ...,
       [  4.80147963,   5.1143589 ],
       [  5.2771781 ,   1.92562627],
       [  7.92820721,   7.30391311]])

In [5]:
arlotta_adata.obsm["X_umap"] = umap_info

### Plotting UMAP

In [ ]:
plt.rcParams['figure.figsize'] = [7, 7]
plt.rcParams['figure.dpi'] = 440

fig, ax = plt.subplots()
sc.pl.umap(
    arlotta_adata,
    color='New_cellType',
    ax = ax,
    title = "Arlotta provided UMAP"
)
ax.axis('equal')

In [ ]:
plt.rcParams['figure.figsize'] = [7, 7]
plt.rcParams['figure.dpi'] = 440

fig, ax = plt.subplots()
sc.pl.umap(
    arlotta_adata,
    color='Gral_cellType',
    ax = ax,
    title = "Arlotta provided UMAP"
)
ax.axis('equal')

In [ ]:
plt.rcParams['figure.figsize'] = [7, 7]
plt.rcParams['figure.dpi'] = 440

fig, ax = plt.subplots()
sc.pl.umap(
    arlotta_adata,
    color='orig_ident',
    palette = 'Spectral',
    ax = ax,
    title = "Arlotta provided UMAP"
)
ax.axis('equal')

## Processing adata

### Removing non-excitatory cells

In [10]:
excitatory = ['Excitatory neurons', 'Apical progenitors', 'Intermediate progenitors']
barcodes_excitatory = arlotta_adata.obs['Gral_cellType'].isin(excitatory)
arlotta_adata = arlotta_adata[barcodes_excitatory]

In [ ]:
plt.rcParams['figure.figsize'] = [7, 7]
plt.rcParams['figure.dpi'] = 440

fig, ax = plt.subplots()
sc.pl.umap(
    arlotta_adata,
    color='New_cellType',
    ax = ax,
    title = "Arlotta provided UMAP (excitatory only)"
)
ax.axis('equal')

### Filtering genes

In [33]:
copy_arlotta_scVelo_default = arlotta_adata.copy()
copy_arlotta_our_filtering = arlotta_adata.copy()

In [15]:
scv.pp.filter_and_normalize(copy_arlotta_scVelo_default, min_shared_counts=20, n_top_genes=3000)
scv.pp.moments(copy_arlotta_scVelo_default, n_pcs=30, n_neighbors=30)

Filtered out 38969 genes that are detected 20 counts (shared).
Normalized count data: X, spliced, unspliced.
Extracted 3000 highly variable genes.
computing neighbors
    finished (0:00:33) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:08) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


In [16]:
copy_arlotta_scVelo_default

AnnData object with n_obs × n_vars = 50323 × 3000
    obs: 'batch', 'orig_ident', 'nCount_RNA', 'nFeature_RNA', 'percent_mito', 'n_hkgene', 'S_Score', 'G2M_Score', 'Phase', 'CC_Difference', 'seurat_clusters', 'RNA_snn_res_1', 'scrublet_doublet', 'RNA_snn_res_2', 'Doublet_intersect', 'Gral_cellType', 'New_cellType', 'biosample_id', 'donor_id', 'species', 'disease', 'disease__ontology_label', 'organ', 'organ__ontology_label', 'library_preparation_protocol', 'library_preparation_protocol__ontology_label', 'sex', 'species__ontology_label', 'initial_size', 'initial_size_unspliced', 'initial_size_spliced', 'n_counts'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'gene_count_corr', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'New_cellType_colors', 'Gral_cellType_colors', 'orig_ident_colors', 'pca', 'neighbors'
    obsm: 'X_umap', 'X_pca'
    varm: 'PCs'
    layers: 'ambiguous', 'matrix', '

In [17]:
scv.tl.velocity(copy_arlotta_scVelo_default)
scv.tl.velocity_graph(copy_arlotta_scVelo_default)

computing velocities
    finished (0:00:20) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
computing velocity graph (using 1/96 cores)


  0%|          | 0/50323 [00:00<?, ?cells/s]

    finished (0:03:55) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)


In [ ]:
scv.pl.velocity_embedding_stream(copy_arlotta_scVelo_default, 
                                 color = 'New_cellType', 
                                 basis='umap',
                                 title="Arlotta w/ default scVelo filtering method")

### Following our own filtering regime

In [34]:
# summing the number of splice features for each cell
sum_spliced_features = np.array(copy_arlotta_our_filtering.layers["spliced"].sum(axis=1))
sum_unspliced_features = np.array(copy_arlotta_our_filtering.layers["unspliced"].sum(axis=1))

# filter out cells with less than 500 spliced or unspliced features
cells_w_enough_splice_info = np.asarray((sum_spliced_features + sum_unspliced_features) >= 500)

copy_arlotta_our_filtering = copy_arlotta_our_filtering[cells_w_enough_splice_info, :]

scv.pp.normalize_per_cell(copy_arlotta_our_filtering, counts_per_cell_after=1e4, enforce=True)
sc.pp.highly_variable_genes(copy_arlotta_our_filtering, flavor='seurat_v3', n_top_genes=3000)
scv.pp.log1p(copy_arlotta_our_filtering)

Normalized count data: X, spliced, unspliced.


In [35]:
copy_arlotta_our_filtering

AnnData object with n_obs × n_vars = 50319 × 48820
    obs: 'batch', 'orig_ident', 'nCount_RNA', 'nFeature_RNA', 'percent_mito', 'n_hkgene', 'S_Score', 'G2M_Score', 'Phase', 'CC_Difference', 'seurat_clusters', 'RNA_snn_res_1', 'scrublet_doublet', 'RNA_snn_res_2', 'Doublet_intersect', 'Gral_cellType', 'New_cellType', 'biosample_id', 'donor_id', 'species', 'disease', 'disease__ontology_label', 'organ', 'organ__ontology_label', 'library_preparation_protocol', 'library_preparation_protocol__ontology_label', 'sex', 'species__ontology_label', 'initial_size', 'n_counts'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'gene_count_corr'
    uns: 'hvg', 'New_cellType_colors', 'Gral_cellType_colors', 'orig_ident_colors'
    obsm: 'X_umap'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'

In [36]:
scv.pp.moments(copy_arlotta_our_filtering, n_pcs=50, n_neighbors=15, use_highly_variable=True)

computing neighbors
    finished (0:00:04) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:26) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


In [ ]:
copy_arlotta_our_filtering

In [27]:
scv.tl.velocity(copy_arlotta_our_filtering, use_highly_variable=True)

computing velocities
    finished (0:04:58) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)


In [28]:
scv.tl.velocity_graph(copy_arlotta_our_filtering)

computing velocity graph (using 1/96 cores)


  0%|          | 0/50319 [00:00<?, ?cells/s]

    finished (0:01:31) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)


In [ ]:
scv.pl.velocity_embedding_stream(copy_arlotta_our_filtering, 
                                 color = 'New_cellType', 
                                 basis='umap',
                                title="Arlotta w/ our own filtering regime")

Note: There does not seem to be any drastic difference between the filtering approaches. Thus, we will continue with our own filtering approach.

## Saving arlotta h5ad w/ UMAP coords

In [31]:
arlotta_adata.write_h5ad(path_to_umap + "arlotta.raw_counts.provided_umap.h5ad")